In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import pyplot, cm
%matplotlib inline

In [ ]:
u = 0.5; d = np.pi/3; beta = 1; Lx = 2.*np.pi
def deff_tuple(sav_file,u,d,beta,Lx,i):
    data = np.load(sav_file)
    try:
        nsav = data['nhst']
    except KeyError:
        nsav = data['nsav']
    D = data['D']; Ly = Lx; nx = data['nx']
    m = data['m']; eta = data['eta']
    nt = data['nt']; dt = data['dt']; dy = data['dx']
    x = np.linspace(0,Ly,nx)
    Pe = (u*d*beta)/D
    Re = eta*(m/12)*(Lx/D)
    navg_f = np.sum(nsav[-1,:,:]*dy,axis=0)/Ly
    navg = navg_f
    navg_i = np.sum(nsav[0,:,:]*dy,axis=0)/Ly
    deltax = x[int(nx*(31/64))]-x[int(nx*(20/64))]
    deltat = dt*(nt)
    deff = (deltax/deltat)*((navg_f[int(nx*(31/64))]-navg_i[int(nx*(31/64))])/(navg_f[int(nx*(31/64))]-navg_f[int(nx*(20/64))]))
    return D,Pe,Re,deff

In [ ]:
m_modes = [0,1,2,3,4,5,6]
dicts = {}
fig = plt.figure(figsize=(12,10))
ax = fig.add_subplot(111,projection='3d')
for k in range(len(m_modes)):
    dicts[k] = [[],[],[]]
    zero_mode = '.\shearflow_savez_0030\shear_updated\pst-n-pe-0030-m-00-alpha-00.npz'
    D,Pe,Re,deff = deff_tuple(zero_mode,u,d,beta,Lx)
    dicts[k][0].append(Pe); dicts[k][1].append(Re); dicts[k][2].append((-1)*deff)
    sav_file = sorted(glob.glob('.\shearflow_savez_0030\shear_updated\pst-n-pe-0030-m-0'+str(m_modes[k])+'-*'))
    for i in range(len(sav_file)):
        D,Pe,Re,deff = deff_tuple(sav_file[i],u,d,beta,Lx)
        dicts[k][0].append(Pe); dicts[k][1].append(Re); dicts[k][2].append((-1)*deff)
    if k == 0:
        ax.scatter(dicts[k][1],dicts[k][0],dicts[k][2],s=50,c='black')
        ax.plot(dicts[k][1],dicts[k][0],dicts[k][2],c='black')
    else:
        ax.scatter(dicts[k][1],dicts[k][0],dicts[k][2])
        ax.plot(dicts[k][1],dicts[k][0],dicts[k][2],linewidth=0.4)
ax.set_xlabel('Re',fontsize=20,labelpad=15)
ax.set_ylabel('Pe',fontsize=20,labelpad=15)
ax.set_zlabel(r'$D^*_*$',fontsize=20,labelpad=15)
# ax.grid(False)

plt.show()

In [ ]:
fig, ax = plt.subplots()
for i in range(len(m_modes)):
    if i == 0:
        ax.scatter(dicts[i][1],dicts[i][2],s=80,c='black',label='m = '+str(m_modes[i]))
    else:
        ax.plot(dicts[i][1],dicts[i][2],label='m = '+str(m_modes[i]))
        ax.scatter(dicts[i][1],dicts[i][2])
ax.set_ylabel(r'$D^*_*$',fontsize=20)
ax.set_xlabel('Re',fontsize=20)
ax.legend(loc='best',shadow=True)
ax.set_ylim(0,0.0175)

plt.tight_layout()
plt.grid()
plt.show()